<a href="https://colab.research.google.com/github/oddrationale/AdventOfCode2023Python/blob/main/Day03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<article class="day-desc"><h2>--- Day 3: Gear Ratios ---</h2><p>You and the Elf eventually reach a <a href="https://en.wikipedia.org/wiki/Gondola_lift" target="_blank">gondola lift</a> station; he says the gondola lift will take you up to the <em>water source</em>, but this is as far as he can bring you. You go inside.</p>
<p>It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.</p>
<p>"Aaah!"</p>
<p>You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.</p>
<p>The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can <em>add up all the part numbers</em> in the engine schematic, it should be easy to work out which part is missing.</p>
<p>The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently <em>any number adjacent to a symbol</em>, even diagonally, is a "part number" and should be included in your sum. (Periods (<code>.</code>) do not count as a symbol.)</p>
<p>Here is an example engine schematic:</p>
<pre><code>467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
</code></pre>
<p>In this schematic, two numbers are <em>not</em> part numbers because they are not adjacent to a symbol: <code>114</code> (top right) and <code>58</code> (middle right). Every other number is adjacent to a symbol and so <em>is</em> a part number; their sum is <code><em>4361</em></code>.</p>
<p>Of course, the actual engine schematic is much larger. <em>What is the sum of all of the part numbers in the engine schematic?</em></p>
</article>

In [1]:
from google.colab import drive
drive.mount(r"/content/drive")

input_file = r"/content/drive/MyDrive/Colab Notebooks/AoC2023/input/03.txt"
with open(input_file, "r") as file:
    input = file.read()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import re

from itertools import product
from typing import List

from pydantic import BaseModel

In [3]:
engine_schematic = [line for line in input.splitlines() if line]

In [4]:
class Number(BaseModel):
    """
    Represents a numeric entity within an engine schematic.

    Attributes:
        value (int): The numerical value of the entity.
        start_x (int): The starting x-coordinate of the entity in the schematic.
        end_x (int): The ending x-coordinate of the entity in the schematic.
        y (int): The y-coordinate (row) where the entity is located in the schematic.
    """

    value: int
    start_x: int
    end_x: int
    y: int

    def is_part_number(self, engine_schematic: List[str]) -> bool:
        """
        Determines whether the number is a "part number".

        This method checks the adjacent cells around the number in the engine schematic. If any adjacent cell contains
        a character that is not a dot or a digit, the method concludes that this number is a part number.

        Args:
            engine_schematic (List[str]): A list of strings representing the engine schematic, where each string
                                          is a row in the schematic.

        Returns:
            bool: True if the number is a part number, False otherwise.
        """

        # Get the dimensions of the engine schematic
        max_y = len(engine_schematic)
        max_x = len(engine_schematic[0])

        # Generate coordinates for all adjacent cells
        adjacent_coordinates = product(
            range(self.y - 1, self.y + 2),
            range(self.start_x - 1, self.end_x + 2)
        )

        # Extract characters from these coordinates, excluding the number itself
        adjacent_characters = [
            engine_schematic[y][x]
            for y, x in adjacent_coordinates
            if 0 <= y < max_y and 0 <= x < max_x
            and (y != self.y or x not in range(self.start_x, self.end_x + 1))
        ]

        # Check if any adjacent character is not a dot or a digit
        return any(c for c in adjacent_characters if c not in ".0123456789")


In [5]:
def find_numbers(engine_schematic: List[str]) -> List[Number]:
    """
    Identifies and extracts numbers from an engine schematic.

    This function scans through each line of the engine schematic, using regular expressions to find
    sequences of digits which represent numbers. For each found number, it creates a 'Number' instance
    recording the number's value and its position in the schematic.

    Args:
        engine_schematic (List[str]): A list of strings representing the engine schematic, where each string
                                      is a row in the schematic.

    Returns:
        List[Number]: A list of 'Number' objects, each representing a number found in the schematic.
    """

    numbers = []
    for y, line in enumerate(engine_schematic):
        # Search for sequences of digits in each line
        for m in re.finditer(r"\d+", line):
            numbers.append(Number(
                value=int(m.group(0)),
                start_x=m.start(),
                end_x=m.end() - 1,
                y=y,
            ))
    return numbers

In [6]:
# Calculate the sum of values of numbers that are a part number
sum([n.value for n in find_numbers(engine_schematic) if n.is_part_number(engine_schematic)])

531932

<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The engineer finds the missing part and installs it in the engine! As the engine springs to life, you jump in the closest gondola, finally ready to ascend to the water source.</p>
<p>You don't seem to be going very fast, though. Maybe something is still wrong? Fortunately, the gondola has a phone labeled "help", so you pick it up and the engineer answers.</p>
<p>Before you can explain the situation, she suggests that you look out the window. There stands the engineer, holding a phone in one hand and waving with the other. You're going so slowly that you haven't even left the station. You exit the gondola.</p>
<p>The missing part wasn't the only issue - one of the gears in the engine is wrong. A <em>gear</em> is any <code>*</code> symbol that is adjacent to <em>exactly two part numbers</em>. Its <em>gear ratio</em> is the result of <span title="They're magic gears.">multiplying</span> those two numbers together.</p>
<p>This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.</p>
<p>Consider the same engine schematic again:</p>
<pre><code>467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
</code></pre>
<p>In this schematic, there are <em>two</em> gears. The first is in the top left; it has part numbers <code>467</code> and <code>35</code>, so its gear ratio is <code>16345</code>. The second gear is in the lower right; its gear ratio is <code>451490</code>. (The <code>*</code> adjacent to <code>617</code> is <em>not</em> a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces <code><em>467835</em></code>.</p>
<p><em>What is the sum of all of the gear ratios in your engine schematic?</em></p>
</article>

In [7]:
class Gear(BaseModel):
    """
    Represents a gear in the engine schematic, defined by two part numbers.

    Attributes:
        part_number_1 (int): The first part number associated with the gear.
        part_number_2 (int): The second part number associated with the gear.
    """

    part_number_1: int
    part_number_2: int

    def gear_ratio(self) -> int:
        """
        Calculates the gear ratio based on the two part numbers.

        Returns:
            int: The product of the two part numbers representing the gear ratio.
        """
        return self.part_number_1 * self.part_number_2

In [8]:
def find_gears(engine_schematic: List[str]) -> List[Gear]:
    """
    Identifies and extracts gears from an engine schematic.

    This function scans the engine schematic for gears, represented by asterisks (*). It identifies the part numbers
    adjacent to each gear and constructs a Gear object for each pair of part numbers found adjacent to a gear.

    Args:
        engine_schematic (List[str]): A list of strings representing the engine schematic, where each string
                                      is a row in the schematic.

    Returns:
        List[Gear]: A list of 'Gear' objects, each representing a gear found in the schematic.
    """

    gears = []
    max_y = len(engine_schematic)
    max_x = len(engine_schematic[0])

    # Iterate through each line and character in the schematic
    for y in range(max_y):
        for m in re.finditer(r"\*", engine_schematic[y]):
            adjacent_part_number = []

            # Check the current and adjacent lines for part numbers near the gear
            for dy in [-1, 0, 1]:
                if 0 <= y + dy < max_y:
                    for n in re.finditer(r"\d+", engine_schematic[y + dy]):
                        if m.start() in range(n.start() - 1, n.end() + 1):
                            adjacent_part_number.append(int(n.group(0)))

            # Create a Gear object if exactly two part numbers are found adjacent to the gear
            if len(adjacent_part_number) == 2:
                gears.append(Gear(part_number_1=adjacent_part_number[0], part_number_2=adjacent_part_number[1]))

    return gears

In [9]:
# Calculate the sum of gear ratios for all gears found in the engine schematic
sum([g.gear_ratio() for g in find_gears(engine_schematic)])

73646890